# Todo

1. 优化函数的输入输出，md介绍文档。

    ~~修改评价类问题的特征创建函数，可以简单一点。~~

2. 能不能更改planner，developer的框架。

    planner也许不需要修改，developer我想要改成不是返回函数列表，而是直接返回代码块，这样后续的处理方式就没那么多限制。

3. 代码报错的critic模块。
    循环三次，如果没成功就可以停止。

## 优化developer

In [1]:
pre_planer ="""
# Role:任务规划师
你是一位高效、严谨的任务规划师，你的任务是根据给定的问题背景和数据情况，制定一份结构清晰、合理和简洁的任务规划。

## Skills
- 理解和分析输入的问题背景和数据情况。
- 将问题按照要求对应为具体的、可执行的任务。

## Packages
你只能用以下用户自定义库进行任务规划，不能使用其他库。每个自定义库包含功能如下：
### data_clean
数据清洗，包括填充缺失值、异常值处理、去除重复值。
### feature_process
特征处理，包括独热编码、标签编码、列映射（map_func_col），跨表生成特征函数（two_data_func_col），
单数据集特征生成（ generate_single_feature ），行特征生成（apply_feature_row），双数据集订单满足率计算（ Order_Fulfillment_cal）。
### machine_learning
- 回归模型：XGBoost,LightGBM,线性,多项式,指数,幂律，RANSAC鲁棒回归。
- 分类模型：XGBoost,LightGBM,逻辑回归，SVM，KNN，决策树。
- 时序模型：ARIMA,LSTM。
### evaluate_model
- 数学建模综合评价类问题的函数，并不是评估模型性能的函数。
- 包含熵权法，TOPSIS，加权评分模型。
### math_optimization
包含线性规划（solve_lp）、整数规划(solve_ilp)、非线性规划（solve_nlp）、二次规划（solve_qp）、
二次凸规划（solve_qcqp）、0-1整数规划(solve_zop)、二阶锥规划(solve_socp)、
多目标规划函数(solve_mulob)，多维背包问题求解函数(multidimensional_knapsack)，二次指派问题(quadratic_assignment)求解函数。
### graph_optimization
包含Dijkstra，最小生成树，papgerenk，解顶点着色问题，解最大流问题函数。
### statistics
统计学函数，假设检验函数，分布检验函数。

## OutputFormat
请严格按照以下 Markdown 格式输出任务规划，：
```markdown
# PLAN
[分析任务背景和问题类型]
## STEP 1
任务: [具体任务描述，说明调用哪个库的哪个函数，以及原因]

## STEP 2
任务: [具体任务描述，说明调用哪个库的哪个函数，以及原因]
...
```
### OutputFormat要求
- 每个STEP只需要明确说明**“因为什么原因所以调用哪个库中的哪个函数”。
- **禁止给出**建模分析描述类型的STEP，只需要具体调用操作的STEP。
- 请注意只能使用`Packages`当中的库和包含的函数，不能使用其他库。
- 数据预处理部分每个STEP只能调用单个函数，不能同时调用多个函数。
- 若为分类问题需要考虑是否对目标变量进行标签编码，但回归问题不需要考虑。
- 模型训练部分只能选择单个模型，不需要额外增加STEP进行模型评估，并且应该在最后的STEP。
- 数据预处理部分并不一定需要，需要自己判断是否需要数据预处理，不需要显示在输出当中。
- STEP结束不需要给出任何提示和注释！
"""

In [1]:
pre_developer="""
# Role: 预测类问题助手

## Skills
1. 能准确解析比赛分析输出中的关键问题。
2. 根据问题分阶段匹配适合的函数（数据预处理或模型训练）。

## Background
为提升比赛分析效率，用户提供了多个函数，用于数据预处理和模型训练任务。助手的任务是根据比赛分析的需求，在不同阶段选择最合适的函数以解决问题。

## Function Categories
<functions>

## Goals
助手的任务分为两个阶段：
### 阶段 1：数据预处理
目标：系统分析和处理原始数据，为模型训练做好准备。
- 分析数据类型、特征分布和缺失值情况。
- 根据需求进行数据清洗和特征提取，注意不要使用待弃用列。
- 如果数据并不需要进行预处理，则直接进入模型训练阶段，比如大部分的时序问题等等。

### 阶段 2：模型训练
目标：构建、训练和评估模型，生成优化的解决方案。
- 确定目标变量和评估指标。
- 选取合适的模型函数进行训练。

### 阶段 3：代码合并
目标：将阶段 1 和阶段 2 的函数调用合并为一个完整的 Python 脚本。
  - 确保代码逻辑清晰，易于理解和维护。
  - 包含必要的注释。

## OutputFormat
```markdown
# 动态函数调用报告

## 1. 识别问题
目标：从比赛分析中提取的关键问题
- ID列名以<ID></ID>包裹。
- 问题类型以<type></type>包裹，选择分类、回归、时间序列当中之一，注意分辨问题。
- 不参与模型训练列名以<drop></drop>包裹，例如<drop>特征1，特征2</drop>。
- 模型的目标变量以<target></target>包裹。

## 2. 阶段 1：数据预处理
- 评估问题是否需要数据预处理，如需数据处理则进入下一步。
- 若不需要数据预处理，则直接进入模型训练阶段。
- 若某阶段未调用函数，不需要输出格式示例函数。
### 2.1 函数调用说明
- **数据预处理函数调用**：明确列出所使用的数据预处理函数及其参数，参数不能自己捏造！
- **处理一致性**：确保训练集和测试集的处理流程一致，方便后续预测使用。
- **函数选择**：根据实际需求选择合适的函数，尽量减少不必要的函数调用，以提升代码效率。
- **列名选择**：不对不参与模型训练的列进行数据预处理。
- **目标变量处理**：必须根据实际情况考虑，因为大多数分类问题需要用`label_encode`对于"train_data"的目标列处理，而回归问题不需要考虑。

格式如下，所有函数包含在[]当中：
```python
[
    {"name": "函数名称", "args": {"参数1": "值1", "参数2": "值2"}},
    {"name": "函数名称", "args": {"参数1": "值1", "参数2": "值2"}}
]
```

## 3. 阶段 2：模型训练
### 3.1 调用的函数
- 调用的模型训练函数及参数，若为分类问题均使用多分类。
- 只能选择单个模型进行训练。

格式如下，所有函数包含在[]当中：
```python
[
    {"name": "函数名称", "args": {"参数1": "值1", "参数2": "值2"}},
    {"name": "函数名称", "args": {"参数1": "值1", "参数2": "值2"}}
]
```

## 4. 阶段3：代码合并
### 4.1 合并代码
- 将阶段1和阶段2的代码合并为一个完整的Python脚本，包含必要的注释。
- 代码不需要导入包，也不需要自己定义函数，只需要保证函数调用正确。
- 如果有数据，应当首先根据路径读取数据。
- 确保代码逻辑清晰，易于理解和维护。
- 代码块应该由```python开头和```结尾。

## Rules
1. 所有输出必须为中文，逻辑清晰。
2. 函数调用严格按照阶段目标，阶段1,2只能调用`Function Categories`当中存在的函数，不得捏造。
3. 针对于`params`参数，必须根据具体情况进行填写，不使用默认值。
4. 在阶段1当中不要对**训练弃用的特征**做处理。
5. 注意输出格式按照模板进行输出，不需要其他注释。
"""

In [4]:
from planner.tools import get_pla_user
from planner.prompt import get_planer
from utils.api import gpt_chat
from utils.utils1 import read_file,write_file,extract_functions,convert_str_function
import os
from planner.tools import get_plan
from utils.rag import ChoromaDBManager
from developer.prompt import get_developer
from developer.tools import get_dev_user
from utils.notebook_serializer import NotebookSerializer
from utils.local_interpreter import LocalCodeInterpreter
import sys
import json
import importlib
import re

/home/zyx/anaconda3/envs/base1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
type1="pre"
question="./test_case/p1/question.txt"
agent ="deepseek"

In [ ]:
# planner环节
print("planner start")
planer = pre_planer
info = get_pla_user(ques=question,problem_type=type1)
response1 = gpt_chat(sys=planer,user=info,provider=agent)
write_file(os.path.join(os.path.dirname(question),f"plan_{agent}.txt"), response1)

planner start


In [8]:
# RAG环节
print("RAG start")
chroma_db = ChoromaDBManager(os.path.join(os.path.dirname(question), "tool_db"))
chroma_db.store_tools_to_db(dir_path="./tool_doc_md")
plan = get_plan(str_path=os.path.join(os.path.dirname(question),f"plan_{agent}.txt"))
prepare_funcs=chroma_db.get_all_tools(plan)

RAG start
✅ 已存储: statistic.md (包含 2 个工具块)
✅ 已存储: data_clean.md (包含 5 个工具块)
✅ 已存储: graph_optimization.md (包含 5 个工具块)
✅ 已存储: machine_learning.md (包含 15 个工具块)
✅ 已存储: math_optimization.md (包含 10 个工具块)
✅ 已存储: feature_process.md (包含 7 个工具块)
✅ 已存储: evaluate_model.md (包含 3 个工具块)


In [73]:
prepare_funcs = [{'tool_name': 'fill_missing_values',
  'content': "**Name:** fill_missing_values  \n**Description:** 填充DataFrame中的缺失值，根据指定的列和填充策略完成缺失值的处理。  \n**Applicable Situations:**  \n- 数据集中存在缺失值时需要处理以确保数据的完整性。  \n- 根据特定策略（如删除、均值填充等）处理缺失值。  \n**Parameters:**  \n- `data`:  \n  - **Type:** `pd.DataFrame`  \n  - **Description:** 包含缺失值需要处理的数据集。  \n\n- `columns`:  \n  - **Type:** `Optional[Union[str, List[str]]]`  \n  - **Description:** 必须提供需要处理的列名,支持单个字符串或列表。如果是`None`，默认检查所有列 \n\n- `strategy`:  \n  - **Type:** `str`  \n  - **Description:** 缺失值的处理策略。默认值为`'auto'`。  \n    - `'auto'`: 删除包含缺失值的行。  \n    - `'mean'`: 使用列的均值填充缺失值，只能用于数值类型的数据列。  \n    - `'median'`: 使用列的中位数填充缺失值，只能用于数值类型的数据列。\n    - `'mode'`: 使用列的众数填充缺失值。  \n\n**Result:**  \n- 返回一个新的DataFrame，其中指定列的缺失值已根据策略进行处理。  \n\n**Notes:**  \n- `columns`默认为`None`检查所有的列。\n- `'auto'`策略，所有包含缺失值的行将被删除,一般不采用。 \n- `'mean'`、`'median'`只能用于数值类型的数据列。",
  'source_file': 'data_clean.md',
  'distance': 0.4805421829223633},
 {'tool_name': 'one_hot_encode',
  'content': '**Name:** one_hot_encode  \n**Description:** 对指定列进行独热编码（One-Hot Encoding），将分类数据转换为数值特征，方便模型输入。  \n\n**Applicable Situations:**  \n- 当数据中包含分类特征，需将其转换为机器学习模型能够处理的数值形式。  \n- 需要减少模型对分类特征排序信息的误解。  \n\n**Parameters:**  \n- `data`:  \n  - **Type:** `pd.DataFrame`  \n  - **Description:** 输入数据集，包含需要编码的列。  \n\n- `columns`:  \n  - **Type:** `Union[str, List[str]]`  \n  - **Description:** 需要进行独热编码的列名，可以是单列名称或列名列表。  \n\n**Result:**  \n- 返回处理后的DataFrame，其中指定的列被替换为独热编码生成的数值列。  \n\n**Notes:**  \n- 输入参数`columns`必须是字符串或字符串列表。  \n- 对于分类列的每个类别，生成一个新列（列名格式为`原列名_类别名`）。  \n- 独热编码会自动忽略原列中的数值特征，因此应确保选择的是分类列。',
  'source_file': 'feature_process.md',
  'distance': 0.5620913505554199},
 {'tool_name': 'label_encode',
  'content': '**Name:** label_encode  \n**Description:** 对指定列进行标签编码（Label Encoding），将分类特征映射为整数值, 为分类任务准备。  \n\n**Applicable Situations:**  \n- 当数据中包含分类特征，需要将其转换为整数值以供机器学习模型处理。  \n- 标签之间具有隐式顺序（如排名数据）时优先使用。  \n\n**Parameters:**  \n- `data`:  \n  - **Type:** `pd.DataFrame`  \n  - **Description:** 输入数据集，包含需要编码的列。  \n\n- `columns`:  \n  - **Type:** `Union[str, List[str]]`  \n  - **Description:** 需要进行标签编码的列名，可以是单列名称或列名列表。\n\n**Result:**  \n- 返回处理后的DataFrame，其中指定的列被替换为整数编码的列。  \n\n**Notes:**  \n- 输入参数`columns`必须是字符串或字符串列表。  \n- 标签编码会将分类列中的每个类别映射到唯一的整数值，范围为`[0, n_classes-1]`。  \n- 对于未处理的分类数据，应优先检查是否有缺失值或不合理类别值。',
  'source_file': 'feature_process.md',
  'distance': 0.6562402248382568},
 {'tool_name': 'train_lightgbm_classifier',
  'content': "**Name:** train_lightgbm_classifier  \n**Description:** 训练LightGBM模型用于分类任务，对新数据进行预测，并输出模型在测试集上的表现（使用Accuracy作为评估指标）。  \n\n**Applicable Situations:**  \n- 适用于分类任务，训练数据为带有目标列的DataFrame。  \n- 需要预测新数据的类别标签输出并返回相应结果。  \n\n**Parameters:**  \n- `data`:  \n  - **Type:** `pd.DataFrame`  \n  - **Description:** 训练的数据集，包括特征和目标列。  \n\n- `target`:  \n  - **Type:** `string`  \n  - **Description:** 目标列名称，模型需要预测的类别标签。  \n\n- `new_data`:  \n  - **Type:** `pd.DataFrame`  \n  - **Description:** 新数据集，用于模型预测。  \n\n- `test_size`:  \n  - **Type:** `float`  \n  - **Description:** 测试集比例，默认为0.2。  \n\n- `params`:  \n  - **Type:** `dict`  \n  - **Description:** LightGBM模型的超参数字典，以下为具体参数举例解释：\n    ```python\n    params = {\n        'objective': 'multiclass',          # 指定分类任务\n        'metric': 'multi_logloss',         # 使用多分类对数损失作为评估指标\n        'num_class': len(np.unique(y_train)),  # 类别数量\n        'max_depth': 6,                    # 树的最大深度\n        'learning_rate': 0.1,              # 学习率\n        'subsample': 0.8,                  # 每次训练使用的样本比例\n        'colsample_bytree': 0.8            # 每次训练使用的特征比例\n    }\n    ```\n\n- `num_boost_round`:  \n  - **Type:** `int`  \n  - **Description:** LightGBM训练的轮次，默认为100。  \n\n- `with_label`:  \n  - **Type:** `bool`  \n  - **Description:** 是否将预测结果与`new_data`一起返回，默认为False。  \n\n**Required:**  \n- `data`  \n- `target`  \n- `new_data`  \n\n**Result:**  \n- 返回预测结果：  \n  - 如果`with_label`为True，则返回包含预测类别的`new_data`；  \n  - 否则，仅返回预测类别的列表。  \n\n**Notes:**  \n- `params`支持用户自定义；若未传入，函数会使用默认超参数配置。  \n- 函数输出模型在测试集上的Accuracy（准确率），帮助用户评估模型性能。",
  'source_file': 'machine_learning.md',
  'distance': 0.5757318735122681}]

In [ ]:
 #developer环节
print("developer start")
# devloper_prompt = get_developer(problem_type=type1,func=prepare_funcs)

describe=""
func = prepare_funcs
for i in func:
    describe+=f"### {i['tool_name']}\n\n"
    describe+=f"{i['content']}\n\n"
    describe+="---\n\n"
 
devloper_prompt = pre_developer.replace("<functions>",describe)
user2 = get_dev_user(question=question,problem_type=type1)
response2 = gpt_chat(sys=devloper_prompt,user=user2,provider=agent)
write_file(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"), response2)
# response2 = read_file(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"))


developer start


In [14]:
import re

def extract_python_code_blocks(file_path):
    with open(file_path, "r") as f:
        md_str = f.read()
    pattern = r"```python(.*?)```"
    matches = re.findall(pattern, md_str, re.DOTALL)
    return [m.strip() for m in matches]
exec_func = extract_python_code_blocks(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"))

In [16]:
exec_func[-1]

'# 读取数据\ntrain_data = pd.read_csv(\'./test_case/p1/train.csv\')\ntest_data = pd.read_csv(\'./test_case/p1/test.csv\')\n\n# 阶段1：数据预处理\n# 填充缺失值\ntrain_data = fill_missing_values(data=train_data, columns=["HomePlanet", "CryoSleep", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], strategy="mode")\ntest_data = fill_missing_values(data=test_data, columns=["HomePlanet", "CryoSleep", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], strategy="mode")\n\n# 独热编码\ntrain_data = one_hot_encode(data=train_data, columns=["HomePlanet", "CryoSleep", "Destination", "VIP"])\ntest_data = one_hot_encode(data=test_data, columns=["HomePlanet", "CryoSleep", "Destination", "VIP"])\n\n# 标签编码目标变量\ntrain_data = label_encode(data=train_data, columns=["Transported"])\n\n# 阶段2：模型训练\npredictions = train_lightgbm_classifier(\n    data=train_data,\n    target="Transported",\n    new_data=test_data,\n    test_size=0.2,\n    params={\n    

In [70]:
notebook = NotebookSerializer("./")
code_interpreter = LocalCodeInterpreter(work_dir="./",notebook_serializer=notebook,task_id="111")
code_interpreter.initialize()

In [74]:
prepare = prepare_funcs
for func in prepare:
    file = set()
    if func["source_file"] not in file:
        module_name = f"tool_code.{func['source_file'].replace('.md', '')}"
        module = importlib.import_module(module_name)
        header = module.get_header()
        tool = f"from tool_code.{func['source_file'].replace('.md', '')} import {func['tool_name']}"
        code_interpreter.execute_code(header)
        code_interpreter.execute_code(tool)
        file.add(func['source_file'])

In [75]:
code = """# 读取数据
train_data = pd.read_csv('./test_case/p1/train.csv')
test_data = pd.read_csv('./test_case/p1/test.csv')

# 阶段1：数据预处理
# 填充缺失值
train_data = fill_missing_values(data=train_data, columns=["HomePlanet", "CryoSleep", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], strategy="mode")
test_data = fill_missing_values(data=test_data, columns=["HomePlanet", "CryoSleep", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], strategy="mode")

# 独热编码
train_data = one_hot_encode(data=train_data, columns=["HomePlanet", "CryoSleep", "Destination", "VIP"])
test_data = one_hot_encode(data=test_data, columns=["HomePlanet", "CryoSleep", "Destination", "VIP"])

# 标签编码目标变量
train_data = label_encode(data=train_data, columns=["Transported"])
"""

In [77]:
code = """predictions = train_lightgbm_classifier(
    data=train_data.drop(["PassengerId", "Cabin", "Name"], axis=1) if train_data.ndim > 1 else train_data,  # 添加维度检查
    target="Transported",
    new_data=test_data.drop(["PassengerId", "Cabin", "Name"], axis=1) if test_data.ndim > 1 else test_data,  # 添加维度检查
    test_size=0.2,
    params={
        "objective": "binary",
        "metric": "binary_logloss",
        "max_depth": 6,
        "num_leaves": 31,  # 添加num_leaves参数
        "learning_rate": 0.1,
        "subsample": 0.8,
        "colsample_bytree": 0.8
    },
    num_boost_round=100,
    with_label=False
)"""

In [80]:
code = """train_data.to_csv('./train_processed.csv', index=False)
test_data.to_csv('./test_processed.csv', index=False)"""

In [81]:
error,need,_=code_interpreter.execute_code(code)

In [82]:
print("error:",error)

error: 


In [1]:
import pandas as pd
train_data = pd.read_csv("./train_processed.csv")
test_data = pd.read_csv("./test_processed.csv")

In [2]:
from tool_code.machine_learning import train_lightgbm_classifier
predictions = train_lightgbm_classifier(
    data=train_data.drop(["PassengerId", "Cabin", "Name"], axis=1) if train_data.ndim > 1 else train_data,  # 添加维度检查
    target="Transported",
    new_data=test_data.drop(["PassengerId", "Cabin", "Name"], axis=1) if test_data.ndim > 1 else test_data,  # 添加维度检查
    test_size=0.2,
    params={
        "objective": "binary",
        "metric": "binary_logloss",
        "max_depth": 6,
        "learning_rate": 0.1,
        "subsample": 0.8,
        "colsample_bytree": 0.8
    },
    num_boost_round=100,
    with_label=False
)

[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=6) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=64) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=6) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=64) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Number of positive: 3472, number of negative: 3482
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.208969 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total

In [35]:
critic_prompt ="""
下面是我的代码和我的报错信息：
<code>{}</code>
报错信息：
<error>{}</error>
请你帮我分析代码错误原因，并且给出修改后的代码，要求：
1. 在原本代码上进行修改，尽可能不增添新的代码。
2. 返回的代码用```python开头和```结尾。
"""

system ="You are a helpful assistant."

In [36]:
user3 = critic_prompt.format(code,error)
response3 = gpt_chat(sys=system,user=user3,provider=agent)
write_file(os.path.join(os.path.dirname(question),f"critic_{agent}.txt"), response3)

In [30]:
user3

'\n下面是我的代码和我的报错信息：\n<code># 阶段2：模型训练\npredictions = train_lightgbm_classifier(\n    data=train_data,\n    target="Transported",\n    new_data=test_data,\n    test_size=0.2,\n    params={\n        "objective": "binary",\n        "metric": "binary_logloss",\n        "max_depth": 6,\n        "learning_rate": 0.1,\n        "subsample": 0.8,\n        "colsample_bytree": 0.8\n    },\n    num_boost_round=100,\n    with_label=False\n)</code>\n报错信息：\n<error>---------------------------------------------------------------------------\nValueError                                Traceback (most recent call last)\nCell In[15], line 2\n      1 # 阶段2：模型训练\n----> 2 predictions = train_lightgbm_classifier(\n      3     data=train_data,\n      4     target="Transported",\n      5     new_data=test_data,\n      6     test_size=0.2,\n      7     params={\n      8         "objective": "binary",\n      9         "metric": "binary_logloss",\n     10         "max_depth": 6,\n  \n... (内容已截断) ...\name}: {pandas_d

# 开源代码

### 第一步planner

In [30]:
from planner.tools import get_pla_user
from planner.prompt import get_planer
from utils.api import gpt_chat
type1="opt"
question="./test_case/o4/question.txt"

planer = get_planer(problem_type=type1)
info = get_pla_user(ques=question,problem_type=type1)

# response1 = gpt_chat(sys=planer,user=info,provider="deepseek")


In [16]:
def write_file(file_path: str, content):
    """
    Write content to a file based on its extension.
    
    Args:
        file_path: Path to the file to be written
        content: Content to write (string for txt/md, list of strings for csv)
    """
    if file_path.endswith('txt'):
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content if isinstance(content, str) else '\n'.join(content))
    elif file_path.endswith('csv'):
        with open(file_path, 'w', encoding='utf-8', newline='') as f:
            if isinstance(content, list):
                f.writelines(content)
            else:
                f.write(content)
    elif file_path.endswith('md'):
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content if isinstance(content, str) else '\n'.join(content))
    else:
        raise ValueError("Unsupported file format")

In [17]:
# 读取文件内容
def read_file(file_path: str):
    """
    Read the content of a file and return it as a string.
    """
    if file_path.endswith('txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    if file_path.endswith('csv'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.readlines()
    if file_path.endswith('md'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

## 匹配函数

In [21]:
from planner.tools import get_plan
from utils.rag import ChoromaDBManager


chroma_db = ChoromaDBManager("./test_case/o8/tool_db")
# chroma_db.store_tools_to_db(dir_path="./tool_doc_md")

In [25]:
os.path.exists(os.path.join(os.path.dirname(question), "tool_db"))

True

In [22]:
plan = get_plan(str_path="./test_case/o8/plan_deepseek.txt")
funcs=chroma_db.get_all_tools(plan)

In [23]:
funcs

[{'tool_name': 'mst',
  'content': '**Name:** mst  \n**Description:** 计算图的最小生成树 (MST)\n**Applicable Situations:**  \n- 网络设计与优化  \n- 电网、通信网络等连接问题  \n- 边权重最小化需求场景  \n\n**Parameters:**  \n- `edges`:  \n  - **Type:** `dict`  \n  - **Description:** 图的边字典，键为 `(u, v)` 的元组，值为对应的边权重  \n- `algorithm`:  \n  - **Type:** `str`  \n  - **Description:** 计算 MST 的算法选择，取值 `\'prim\'` 或 `\'kruskal\'`（默认 `\'prim\'`）  \n\n**Result:**  \n- 返回一个元组 `(mst_edges, total_weight)`  \n  - `mst_edges`: MST 的边列表，每个边的格式为 `(u, v, {\'weight\': w})`  \n  - `total_weight`: MST 的总权重  \n\n**Example Call:**  \n```python\nmst_edges, total_weight = mst(\n    edges={("A", "B"): 1, ("B", "C"): 2, ("A", "C"): 4},\n    algorithm=\'prim\'\n)\n```',
  'source_file': 'graph_optimization.md',
  'distance': 0.6779873371124268}]

## Developer

In [34]:
from developer.prompt import get_developer
from developer.tools import get_dev_user
funcs = [{'tool_name': 'solve_lp',
  'content': '**Name:** solve_lp  \n**Description:** 用于简单线性规划问题，线性规划求解函数。\n**Applicable Situations:**  \n- 需要求解目标函数最小化的线性规划问题  \n\n**Parameters:**  \n- `c`:  \n  - \u200b**Type:** `array-like`  \n  - \u200b**Description:** 目标函数系数向量，维度需与变量数一致  \n\n- `A_ub`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 不等式约束系数矩阵（默认None），每行对应一个约束条件  \n\n- `b_ub`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 不等式约束右侧常数向量（默认None）  \n\n- `A_eq`:  \n  - \u200b**Type:** `2D array-like | None`  \n  - \u200b**Description:** 等式约束系数矩阵（默认None）  \n\n- `b_eq`:  \n  - \u200b**Type:** `array-like | None`  \n  - \u200b**Description:** 等式约束右侧常数向量（默认None）  \n\n- `bounds`:  \n  - \u200b**Type:** `list of tuples | None`  \n  - \u200b**Description:** 变量边界列表（默认None），每个元组表示变量下界和上界  \n  - \u200b**Example:** `[(0, None), (0, 5)]` 表示x₁≥0，x₂∈[0,5]  \n\n**Result:**  \n- 成功时返回包含最优解x和最优值fun的元组  \n- 失败时返回包含错误信息的字符串  \n\n**Example Call:**  \n```python\n# 求解 min -x1 + 4x2 \n# s.t.  x1 + x2 <= 5\n#       2x1 + x2 = 4\n#       x1 ∈ [0,3], x2 ≥ 0\nx_opt, f_opt = solve_lp(\n    c=[-1, 4],\n    A_ub=[[1, 1]],\n    b_ub=[5],\n    A_eq=[[2, 1]],\n    b_eq=[4],\n    bounds=[(0, 3), (0, None)]\n)\n```',
  'source_file': 'math_optimization.md',
  'distance': 0.5363641381263733}]
devloper_prompt = get_developer(problem_type=type1,func=funcs)
user2 = get_dev_user(question=question,problem_type=type1)

# response2 = gpt_chat(sys=devloper_prompt,user=user2)

## 编译

In [35]:
from utils.notebook_serializer import NotebookSerializer
from utils.local_interpreter import LocalCodeInterpreter
import sys
import json
import os
dirname = "./"
notebook = NotebookSerializer(dirname)

# agents code执行
code_interpreter = LocalCodeInterpreter(work_dir=dirname,notebook_serializer=notebook,task_id="111")
code_interpreter.initialize()

In [36]:
import importlib

for func in funcs:
    file = set()
    if func["source_file"] not in file:
        module_name = f"tool_code.{func['source_file'].replace('.md', '')}"
        module = importlib.import_module(module_name)
        header = module.get_header()
        tool = f"from tool_code.{func['source_file'].replace('.md', '')} import {func['tool_name']}"
        code_interpreter.execute_code(header)
        code_interpreter.execute_code(tool)
        file.add(func['source_file'])

In [37]:
from utils.utils1 import extract_functions

functions = extract_functions("./test_case/o4/dev_deepseek.txt")

In [6]:
import ast
def convert_str_function(json_str):
    data = ast.literal_eval(json_str)
    results = []
    for item in data:
        name = item['name']
        args = item['args']
        arg_strs = []
        for k, v in args.items():
            # 保证字典等非字符串不被误转义为字符串
            if isinstance(v, str):
                arg_strs.append(f'{k}="{v}"')
            else:
                arg_strs.append(f"{k}={v}")
        arg_str = ", ".join(arg_strs)
        call_str = f'result = {name}({arg_str})'
        results.append(call_str)

    return "\n".join(results)

### 两步

In [8]:
import re
func_call = convert_str_function(functions[0])
data_processed = func_call.split("\n")
updated_lines = []

train = 0
test = 0

for tmp in data_processed:
    if 'data="train_data"' in tmp:
        if train == 0:
            data_name = 'train_data'
        else:
            data_name = f'train_data{train}'
        output_var = f'train_data{train + 1}'
        tmp = tmp.replace('data="train_data"', f'data={data_name}')
        tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
        train += 1
    elif 'data="test_data"' in tmp:
        if test == 0:
            data_name = 'test_data'
        else:
            data_name = f'test_data{test}'
        output_var = f'test_data{test + 1}'
        tmp = tmp.replace('data="test_data"', f'data={data_name}')
        tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
        test += 1
    updated_lines.append(tmp)
    

In [9]:
updated_lines

['train_data1 = fill_missing_values(data=train_data, columns=[\'HomePlanet\', \'CryoSleep\', \'Destination\', \'Age\', \'VIP\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], strategy="mode")',
 'test_data1 = fill_missing_values(data=test_data, columns=[\'HomePlanet\', \'CryoSleep\', \'Destination\', \'Age\', \'VIP\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], strategy="mode")',
 'train_data2 = detect_and_handle_outliers_zscore(data=train_data1, columns=[\'Age\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], threshold=3.0, method="clip")',
 'test_data2 = detect_and_handle_outliers_zscore(data=test_data1, columns=[\'Age\', \'RoomService\', \'FoodCourt\', \'ShoppingMall\', \'Spa\', \'VRDeck\'], threshold=3.0, method="clip")',
 "train_data3 = one_hot_encode(data=train_data2, columns=['HomePlanet', 'CryoSleep', 'Destination', 'VIP'])",
 "test_data3 = one_hot_encode(data=test_data2, columns=['HomePlanet', 'CryoS

In [10]:
data_load ="""train_data = pd.read_csv("{train_path}")
test_data = pd.read_csv("{test_path}")"""
path1 = "./test_case/p1/train.csv"
path2 = "./test_case/p1/test.csv"
data_load=data_load.format(train_path=path1,test_path=path2)

code_interpreter.execute_code(data_load)

('', False, '')

In [11]:
for i in updated_lines:
    code_interpreter.execute_code(i)

In [14]:
model = convert_str_function(functions[1])
model1=model.replace('"train_data"', f'train_data{train}').replace('"test_data"', f'test_data{test}')

In [15]:
code_interpreter.execute_code(model1)

('---------------------------------------------------------------------------\nKeyError                                  Traceback (most recent call last)\nFile ~/anaconda3/envs/base1/lib/python3.10/site-packages/xgboost/data.py:407, in pandas_feature_info(data, meta, feature_names, feature_types, enable_categorical)\n    406 try:\n--> 407     new_feature_types.append(_pandas_dtype_mapper[dtype.name])\n    408 except KeyError:\n\nKeyError: \'object\'\n\nDuring handling of the above exception, another excepti\n... (内容已截断) ...\n3.10/site-packages/xgboost/data.py:372, in _invalid_dataframe_dtype(data)\n    370 type_err = "DataFrame.dtypes for data must be int, float, bool or category."\n    371 msg = f"""{type_err} {_ENABLE_CAT_ERR} {err}"""\n--> 372 raise ValueError(msg)\n\nValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:PassengerId

### 单步

In [39]:
functions[0]

'[    {        "name": "solve_lp",        "args": {            "c": [-500, -550, -630, -1000, -800, -700, -800, -700, -600, -950, -900, -930, -1000, -960, -840, -650, -600, -700, -1200, -1040, -980, -860, -880, -780],            "A_ub": [                [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],                [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],                [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],                [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],                [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],                [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 

In [38]:
model = convert_str_function(functions[0])
model

ValueError: malformed node or string on line 1: <ast.BinOp object at 0x7f74b7851360>

In [26]:
code_interpreter.execute_code(model)

('', False, '')

## 汇总

In [29]:
from planner.tools import get_pla_user
from planner.prompt import get_planer
from utils.api import gpt_chat
from utils.utils1 import read_file,write_file,extract_functions,convert_str_function
import os
from planner.tools import get_plan
from utils.rag import ChoromaDBManager
from developer.prompt import get_developer
from developer.tools import get_dev_user
from utils.notebook_serializer import NotebookSerializer
from utils.local_interpreter import LocalCodeInterpreter
import sys
import json
import importlib
import re

def func_preprocess(functions):
    func_call = convert_str_function(functions[0])
    data_processed = func_call.split("\n")
    updated_lines = []

    train = 0
    test = 0

    for tmp in data_processed:
        if 'data="train_data"' in tmp:
            if train == 0:
                data_name = 'train_data'
            else:
                data_name = f'train_data{train}'
            output_var = f'train_data{train + 1}'
            tmp = tmp.replace('data="train_data"', f'data={data_name}')
            tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
            train += 1
        elif 'data="test_data"' in tmp:
            if test == 0:
                data_name = 'test_data'
            else:
                data_name = f'test_data{test}'
            output_var = f'test_data{test + 1}'
            tmp = tmp.replace('data="test_data"', f'data={data_name}')
            tmp = re.sub(r'^result\s*=', f'{output_var} =', tmp)
            test += 1
        updated_lines.append(tmp)
    return updated_lines,train,test

def code_exe(prepare,type,functions,question):
    notebook = NotebookSerializer("./")
    code_interpreter = LocalCodeInterpreter(work_dir="./",notebook_serializer=notebook,task_id="111")
    code_interpreter.initialize()

    for func in prepare:
        file = set()
        if func["source_file"] not in file:
            module_name = f"tool_code.{func['source_file'].replace('.md', '')}"
            module = importlib.import_module(module_name)
            header = module.get_header()
            tool = f"from tool_code.{func['source_file'].replace('.md', '')} import {func['tool_name']}"
            code_interpreter.execute_code(header)
            code_interpreter.execute_code(tool)
            file.add(func['source_file'])
    if (len(functions) > 1) and (type in ["pre","eval"]):
        update_func,train_index,test_index = func_preprocess(functions)
        data_load ="""train_data = pd.read_csv("{train_path}")
        test_data = pd.read_csv("{test_path}")"""
        path1 = os.path.join(os.path.dirname(question),"train.csv")
        path2 = os.path.join(os.path.dirname(question),"test.csv")
        data_load=data_load.format(train_path=path1,test_path=path2)
        code_interpreter.execute_code(data_load)
        for i in update_func:
            code_interpreter.execute_code(i)
        model = convert_str_function(functions[1])
        model1=model.replace('"train_data"', f'train_data{train_index}').replace('"test_data"', f'test_data{test_index}')
        code_interpreter.execute_code(model1)
    else:
        model = convert_str_function(functions[0])
        code_interpreter.execute_code(model)
        code_interpreter.execute_code("print(result)")



if __name__ == "__main__":
    type1="opt"
    question="./test_case/o4/question.txt"
    agent ="deepseek"
    # planner环节
    print("planner start")
    planer = get_planer(problem_type=type1)
    info = get_pla_user(ques=question,problem_type=type1)
    # response1 = gpt_chat(sys=planer,user=info,provider="deepseek")
    # write_file(os.path.join(os.path.dirname(question),f"plan_{agent}.txt"), response1)
    response1 = read_file(os.path.join(os.path.dirname(question),f"plan_{agent}.txt"))

    # RAG环节
    print("RAG start")
    chroma_db = ChoromaDBManager(os.path.join(os.path.dirname(question), "tool_db"))
    chroma_db.store_tools_to_db(dir_path="./tool_doc_md")
    plan = get_plan(str_path=os.path.join(os.path.dirname(question),f"plan_{agent}.txt"))
    prepare_funcs=chroma_db.get_all_tools(plan)

    #developer环节
    print("developer start")
    devloper_prompt = get_developer(problem_type=type1,func=prepare_funcs)
    user2 = get_dev_user(question=question,problem_type=type1)
    # response2 = gpt_chat(sys=devloper_prompt,user=user2)
    # write_file(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"), response2)
    response2 = read_file(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"))

    # 代码执行环节
    print("code execute start")
    exec_func = extract_functions(os.path.join(os.path.dirname(question),f"dev_{agent}.txt"))
    code_exe(prepare=prepare_funcs,type=type1,functions=exec_func,question=question)


planner start
RAG start
✅ 已存储: statistic.md (包含 2 个工具块)
✅ 已存储: data_clean.md (包含 5 个工具块)
✅ 已存储: graph_optimization.md (包含 5 个工具块)
✅ 已存储: machine_learning.md (包含 15 个工具块)
✅ 已存储: math_optimization.md (包含 10 个工具块)
✅ 已存储: feature_process.md (包含 7 个工具块)
✅ 已存储: evaluate_model.md (包含 3 个工具块)
developer start
code execute start


ValueError: malformed node or string on line 1: <ast.BinOp object at 0x7f76d8167880>